In [1]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.7 MB/s eta 0:00:00


**Hugginface loggin for push on Hub**

In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [3]:
from datasets import load_dataset

dataset = load_dataset("PlanTL-GOB-ES/cantemist-ner")
dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/19397 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/18165 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11168 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 19397
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 18165
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 11168
    })
})

In [5]:
dataset_train = dataset['train']

In [12]:
dataset_train[0]['tokens']

['ANAMNESIS',
 'Mujer',
 'de',
 '67',
 'años',
 'con',
 'antecedentes',
 'personales',
 'de',
 'hipotiroidismo',
 'en',
 'tratamiento',
 'con',
 'levotiroxina',
 'y',
 'fumadora',
 'activa',
 'de',
 '12.5',
 'paquetes',
 '/',
 'año',
 '.']

In [13]:
from functools import reduce
reduce(lambda a, b: a + " "+ b, dataset_train[0]['tokens'], "")

' ANAMNESIS Mujer de 67 años con antecedentes personales de hipotiroidismo en tratamiento con levotiroxina y fumadora activa de 12.5 paquetes / año .'

In [20]:
train_tags = [x for x in dataset_train if reduce(lambda a,b: a != 0 or b != 0, x['ner_tags']) == True]

In [35]:
def show_ner_tag(item):
  ner_tags = item['ner_tags']
  tokens = item['tokens']
  list_post = [x for x in range(len(ner_tags)) if ner_tags[x] != 0]
  type_list = [tokens[x] for x in list_post]
  print(type_list)
  return reduce(lambda a, b: a + " "+ b, type_list, "")


In [23]:
train_tags[10]

{'id': '64',
 'tokens': ['Descartándose',
  'por',
  'RMN',
  'afectación',
  'metastásica',
  'a',
  'nivel',
  'pancreático',
  '.'],
 'ner_tags': [0, 0, 0, 0, 1, 0, 0, 0, 0]}

In [37]:
from datasets import load_dataset

dataset_CODING = load_dataset("bigbio/cantemist")

/usr/local/lib/python3.10/dist-packages/datasets/load.py:1461: FutureWarning: The repository for bigbio/cantemist contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/bigbio/cantemist
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

In [38]:
dataset_CODING

DatasetDict({
    train: Dataset({
        features: ['id', 'document_id', 'text', 'labels', 'text_bound_annotations', 'events', 'relations', 'equivalences', 'attributes', 'normalizations', 'notes'],
        num_rows: 501
    })
    test: Dataset({
        features: ['id', 'document_id', 'text', 'labels', 'text_bound_annotations', 'events', 'relations', 'equivalences', 'attributes', 'normalizations', 'notes'],
        num_rows: 300
    })
    validation: Dataset({
        features: ['id', 'document_id', 'text', 'labels', 'text_bound_annotations', 'events', 'relations', 'equivalences', 'attributes', 'normalizations', 'notes'],
        num_rows: 500
    })
})

In [85]:
royalListOfCode = {}
with open('ICD-O-3_valid-codes.txt',encoding='utf8') as file:
  linesInFile = file.readlines()
  for iLine in linesInFile:
    listOfData = iLine.split('\t')

    code = listOfData[0]
    description = reduce(lambda a, b: a + " "+ b, listOfData[1:2], "")
    royalListOfCode[code.strip()] = description.strip()
    #if "" != description:
      #print("Code: " + code )
      #print("Description: " + description )
    #listOfCode['code'].append(iLine[0])
    #listOfCode['description'].append(reduce(lambda a, b: a + " "+ b, iLine[1:], ""))
#print (royalListOfCode)


In [91]:
def getCodeDescription(labels_of_type, royalListOfCode):
  classification = []
  for iValue in labels_of_type:
    if iValue in  royalListOfCode.keys():
      classification.append(royalListOfCode[iValue])
  return classification

In [92]:
dataset_train_CODING = dataset_CODING['train']
for item in dataset_train_CODING:
  idFile = item['id']
  text = item['text']
  list_of_type = item['text_bound_annotations']
  labels_of_type = item['labels']
  diagnostyc_types = getCodeDescription( labels_of_type, royalListOfCode)
  print('The file with ID: ' + idFile + ' has clasifications ')
  for iTypes in diagnostyc_types:
    print(iTypes)


The file with ID: 1 has clasifications 
Carcinoma de células pequeñas, SAI
Neoplasia metastásica
The file with ID: 3 has clasifications 
Neoplasia metastásica
Neoplasia de benignidad o malignidad incierta
Carcinoma indiferenciado, SAI
Carcinoma indiferenciado, SAI, metástasis
Neoplasia maligna
The file with ID: 5 has clasifications 
Carcinoma ductal infiltrante, SAI
Neoplasia metastásica
Neoplasia de benignidad o malignidad incierta
Linfoma maligno difuso tipo B de células grandes, SAI

The file with ID: 7 has clasifications 
Neoplasia metastásica
Carcinoma de células grandes, SAI
Carcinoma, no de células pequeñas
Células tumorales malignas
Neoplasia maligna
Neoplasia de benignidad o malignidad incierta
Carcinoma de células grandes, SAI, metástasis
The file with ID: 9 has clasifications 
Neoplasia de benignidad o malignidad incierta
Carcinoma de células transicionales, SAI - grado I, bien diferenciado
Tumor carcinoide, SAI
Mesotelioma epitelioide maligno
Carcinoma neuroendocrino de cél

In [93]:
# royalListOfCode[iValue]
# for iValue in labels_of_type:
#     if iValue in  royalListOfCode.keys():
#         print('Code ==> ', iValue)
#         print('Classification ==> ',royalListOfCode[iValue])
